In [1]:
import json
spec = json.load(open('gpsys.json'))

In [83]:
import numpy as np
import scipy.stats
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process import GaussianProcessRegressor

In [84]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/lukas/Code/pyhfdev/venv/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['mod', 'plot']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [107]:
class mod(object):
    def get_gp(self, X, y, alpha=10**-3, kernel_name='const_rbf', seed = 1234):
        kernel = ConstantKernel(
    #             constant_value_bounds=[1.,100.0]
                ) * Matern(
                    length_scale_bounds=[2.,10.], length_scale = [10.,10.,1.,1.],
                ) + WhiteKernel(noise_level_bounds=[1.,3.]
    #                 constant_value_bounds=[0,0.001]
    #             ) * Matern(
    #                 length_scale_bounds=[.001,100]
                )
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=alpha, random_state=seed)
        gp.fit(X, y.ravel())
        return gp

    def make_gp(self):
        for_gp_X = self.plcmtX[np.asarray(self.evals[:,0], dtype='int')]
        for_gp_X = np.concatenate([for_gp_X,self.evals[:,1:-1]], axis=1)
        for_gp_y = self.evals[:,-1]
        self.gp = self.get_gp(for_gp_X,for_gp_y)    

    def eval_gp(self,chan_ind,alphas,**kwargs):
        evalX  = self.plcmtX[chan_ind]
        evalA  = np.einsum('x,a->xa',np.ones((len(evalX),)),alphas)
        return self.gp.predict(np.concatenate([evalX,evalA],axis=1), **kwargs)

    def __init__(self,s):
        noms = [s['channels'][i]['samples'][0]['data'] for i in range(len(s['channels']))]
        nom = np.concatenate(noms)

        self.evals = np.concatenate([spec['channels'][i]['samples'][0]['modifiers'][0]['data']['evals'] for i in range(len(spec['channels']))])

        cuts = [0] + np.cumsum([len(n) for n in noms]).tolist()
        self.sli = [slice(cuts[i],cuts[i+1]) for i in range(len(cuts)-1)]

        self.chan_ind = list(range(len(nom)))
        self.plcmtX   = np.concatenate([s['channels'][i]['samples'][0]['modifiers'][0]['data']['plcmt'] for i in range(len(s['channels']))])
        self.make_gp()
        
        self.auxdata_deltas = [0.] * len(self.chan_ind)
        self.auxdata_alphas = [0.]*(m.evals.shape[-1]-2)

In [108]:
m = mod(spec)
[0.]*(m.evals.shape[-1]-2)

[0.0, 0.0]

In [390]:
def plot(axarr,m,alphas,gammas):
#     alphas = np.asarray([a0,a1])
#     gammas = np.asarray([global_gamma]*len(m.chan_ind))

    f,axarr = plt.subplots(1,5)
    f.set_size_inches(20,4)

    vmin,vmax = 40,60 
    
    evalX  = m.plcmtX[m.chan_ind]
    mean,err = m.eval_gp(m.chan_ind,alphas, return_std=True)

    deltas = (gammas-1)*mean

    ax = axarr[0]
    ax.set_title('delta constraints')
    ax.set_ylim(-10,10)
    ax.fill_between(m.chan_ind,-err,err, facecolor = 'grey', alpha = 0.2)
    ax.scatter(m.chan_ind,m.auxdata_deltas, c = 'k')
    ax.scatter(m.chan_ind,deltas, c = 'r')
    
    
    ax = axarr[1]
    ax.set_title('channels in X space')
    ax.set_xlim(0,10)
    ax.set_ylim(0,10)
    ax.scatter(evalX[:,0],evalX[:,1], c = mean, vmin = vmin,vmax = vmax)
    
    ax = axarr[2]
    ax.set_title('channels concatenated')
    for sl in m.sli:
        bin_num = m.chan_ind[sl]
        ch_y = mean[sl]

        ch_deltas_applied = (mean+deltas)[sl]

        y_m_err = (mean-err)[sl]
        y_p_err = (mean+err)[sl]
        ax.fill_between(bin_num,y_m_err,y_p_err, facecolor = 'grey', alpha = 0.2)
        ax.scatter(bin_num,ch_y)
        ax.plot(bin_num,ch_y)

        ax.scatter(bin_num,ch_deltas_applied, c = 'k')
        ax.plot(bin_num,ch_deltas_applied, c = 'k')
    ax.set_ylim(0,1.2*vmax)
    
    
    ax = axarr[3]
    ax.set_title('alpha constraint')


    A = x,y = np.mgrid[-1.5:1.5:21j,-1.5:1.5:21j]
    A = np.einsum('abc->bca',A)
    A = A.reshape(21*21,2)
        
    pdf = scipy.stats.multivariate_normal(alphas,[[1.,0],[0,1.]])
    z = pdf.pdf(A)
    z = z.reshape(21,21)

    ax.contourf(x,y,z, cmap = 'viridis')
    ax.scatter(m.auxdata_alphas[0],m.auxdata_alphas[1], c = 'k')
    ax.scatter(alphas[0],alphas[1], c = 'r')

    al = m.evals[:,1:-1]
    ax.scatter(al[:,0],al[:,1], alpha = 0.5, c = 'w')
    ax.set_xlim(-1.5,1.5)
    ax.set_ylim(-1.5,1.5)
    
    
    ax = axarr[4]
    ax.set_title('Evaluations Per Bin')
    ax.scatter(m.evals[:,0],m.evals[:,-1], alpha = 0.1, c = 'b')
    ax.set_xlim(0,np.max(m.chan_ind))
    for sl in m.sli:
        bin_num = m.chan_ind[sl]
        ch_y = mean[sl]


        y_m_err = (mean-err)[sl]
        y_p_err = (mean+err)[sl]
        ax.fill_between(bin_num,y_m_err,y_p_err, facecolor = 'grey', alpha = 0.2)
        ax.plot(bin_num,ch_y, c = 'k', alpha = 0.1)

    ax.set_ylim(0,1.2*vmax)

In [391]:
# from ipywidgets import interact, fixed, widgets
# interact(plot, m = fixed(m), a0 = (-1,1,0.1), a1 = (-1,1,0.1), global_gamma = (0.9,1.1,0.01))

In [404]:
alphas = [widgets.FloatSlider(
    value=np.random.uniform(-0.3,0.3),
    min=-1.5,
    max=1.5,
    step=0.05,
    disabled=False,
    continuous_update=False,
    readout=True,
    readout_format='.1f',
#     layout={'width': '20px', 'height': '130px', 'margin': '1px'}
) for x in m.auxdata_alphas]

gammas = [widgets.FloatSlider(
    value=1 + 0.05*np.sin(i/2.),
    min=0.9,
    max=1.1,
    step=0.01,
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.1f',
    layout={'width': '25px', 'height': '175px', 'margin': '1px'}
) for i,x in enumerate(m.auxdata_deltas)]

controls = widgets.VBox(alphas + [widgets.HBox(gammas)])

In [405]:
def f(m,**kwargs):
    alphas = np.asarray([kwargs['a{}'.format(i)] for i in range(len(m.auxdata_alphas))])
    gammas = np.asarray([kwargs['g{}'.format(i)] for i in range(len(m.auxdata_deltas))])
    plot(axarr,m,alphas,gammas)
    plt.show()
what = {}
for i,a in enumerate(alphas):
    what['a{}'.format(i)] = a
for i,g in enumerate(gammas):
    what['g{}'.format(i)] = g
what['m'] = fixed(m)
out = widgets.interactive_output(f,what)

controls = widgets.VBox(alphas + [widgets.HBox(gammas)])
display(controls)
display(out)

Output()